In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

In [2]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchbearer import Trial
from torchvision.datasets import ImageFolder
import os
import json

!unzip mixed_same.zip

In [122]:
def getDataLoader(datadir = 'mixed_same/val'):
    # convert each image to tensor format
    preprocess = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    #datadir = '/Volumes/DRIVE/test_sets/mixed_same/val'
    #datadir = os.path.join('/Volumes/DRIVE/test_sets/mixed_same/', 'val')
    imgFolder = ImageFolder(datadir, transform=preprocess)
    testloader = DataLoader(imgFolder, batch_size=32, shuffle=False)
    return testloader

In [123]:
def getModelAccuracy(model, testloader, in_to_in9):
    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    for inputs, labels in testloader:
        outputs = model(inputs)

        for (output, real_label) in zip(outputs, labels):
            model_output = in_to_in9[str(torch.argmax(output).item())]

            if model_output == real_label.item():
                correct += 1
            total += 1


    print('Test Accuracy: %2.2f %%' % ((100.0 * correct) / total))

In [124]:
def getIn2In9(root = ''):
    with open(root + 'in_to_in9.json') as f:
      in_to_in9 = json.load(f)
    return in_to_in9

In [3]:
#Import the imagenet pretrained mobilenetv2 model
mobilenet = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
#mobilenet.to('cuda:0')
mobilenet.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /Users/mattwear/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /Users/mattwear/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
#Get the mixed_same dataset
mixed_same = getDataLoader(datadir = 'mixed_same/val')

In [120]:
#Get the imagenet to in9l class converter
in_to_in9 = getIn2In9()

In [116]:
#Compute model accuracy on mixed_same
getModelAccuracy(mobilenet, mixed_same, in_to_in9)

80.04938271604938

In [125]:
#Get the mixed_rand dataset
mixed_rand = getDataLoader(datadir = 'mixed_rand/val')

In [128]:
#Compute model accuracy on mixed_same
getModelAccuracy(mobilenet, mixed_rand, in_to_in9)

Test Accuracy: 70.49 %


Using mobilenetv2 we get a test accuracy of 80.05% on mixed_same and 70.49% on mixed_rand giving a BG-GAP of 9.56%

In [130]:
#Get the original dataset
original = getDataLoader(datadir = 'original/val')

In [131]:
#Compute model accuracy on original
getModelAccuracy(mobilenet, original, in_to_in9)

Test Accuracy: 93.80 %


In [132]:
#Get the only_bg_t dataset
only_bg_t = getDataLoader(datadir = 'only_bg_t/val')

In [133]:
#Compute model accuracy on only_bg_t
getModelAccuracy(mobilenet, only_bg_t, in_to_in9)

Test Accuracy: 13.16 %
